In [4]:
import sys
sys.path.append('../') 

In [5]:
%load_ext autoreload
%autoreload 2
import sklearn
import copy
import numpy as np

import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
# from viz import viz
from bokeh.plotting import figure, show, output_notebook, output_file, save
from functions import merge_data
from sklearn.model_selection import RandomizedSearchCV
import load_data


from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from fit_and_predict import fit_and_predict

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [44]:
preds_df = pd.read_pickle("multi_day_2.pkl")

In [ ]:
def l1(arr1,arr2):
    return sum([np.abs(a1-a2) for (a1,a2) in zip(arr1,arr2)])/len(arr1)

In [47]:
outcome = np.array([preds_df['deaths'].values[i][-1] for i in range(len(preds_df))])
residuals = {}
for days_ahead in [1, 2, 3, 7]:
    for lower_threshold in [0, 10]:
        colname = f'{days_ahead} day, deaths>={lower_threshold}'
        residuals[colname] = []
        for method in ['exponential', 'shared_exponential', 'ensemble']:
            key = f'predicted_deaths_{method}_{days_ahead}'
            preds = [p[-1] for p in preds_df[key][outcome > lower_threshold]]
            residuals[colname].append(l1(outcome[outcome > lower_threshold],preds))
            if method == 'shared_exponential':
                key = f'predicted_deaths_{method}_{days_ahead}_demographics'
                preds = [p[-1] for p in preds_df[key][outcome > lower_threshold]]
                residuals[colname].append(l1(outcome[outcome > lower_threshold],preds))    

In [81]:
def highlight_max(s):
    '''
    highlight the maximum in a Series yellow.
    '''
    is_max = s == s.min()
    return ['background-color: yellow' if v else '' for v in is_max]

In [83]:
res_df = pd.DataFrame(residuals, index=['exponential', 'shared', 'demographics', 'ensemble'])
res_df = res_df.astype(float).round(2)
res_df.style.highlight_min().format("{:.2f}")

,"1 day, deaths>=0","1 day, deaths>=10","2 day, deaths>=0","2 day, deaths>=10","3 day, deaths>=0","3 day, deaths>=10","7 day, deaths>=0","7 day, deaths>=10"
exponential,1.56,14.11,2.27,17.24,4.67,43.65,5.60,54.47
shared,1.81,7.25,2.05,12.74,2.51,20.53,4.12,42.34
demographics,2.12,8.70,2.32,11.71,2.65,19.20,5.08,20.90
ensemble,1.55,10.35,1.53,7.60,2.32,14.85,3.57,33.82


In [1]:
import plotly.express as px

In [28]:
preds_df = preds_df[preds_df.outcome > 10]
preds_df[outcome] = outcome
fig = px.scatter(preds_df, x="outcome", y=f'predicted_deaths_ensemble_3')

In [29]:
fig.show()

In [ ]:
preds_dict = {}
for d in [1, 2, 3, 7]:
    fit_and_predict(train_df, 
                    test_df, 
                    outcome=outcome_type, 
                    mode='eval_mode',
                    method='ensemble',
                    demographic_vars=[],
                    target_day=np.array([d]))
    preds_dict[d] = np.array([p[-1] for p in test_df[f'predicted_deaths_ensemble_{d}']])
preds_dict['FIPS'] = test_df['countyFIPS']
preds_dict['outcome'] = np.array([df[outcome_type].values[i][-1] for i in range(len(df))])
preds_dict['county'] = df['CountyNamew/StateAbbrev'].values
preds_df = pd.DataFrame(preds_dict)
preds_df.to_pickle('multi_day_1.pkl')